This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

## Setting up environment

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "voc-183321194312667737058046757cd49b1d8b6.91865541"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [2]:
pip install chromadb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sentence-transformers==2.2.2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Libraries
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = OpenAI(model_name=model_name, temperature=temperature, max_tokens = 2000)

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


## Synthetic Data Generation
#### Generating Real Estate Listings with an LLM
use a Large Language Model (LLM) to generate at least 10 diverse and realistic real estate listings containing facts about the real estate.

In [5]:
import json
from sentence_transformers import SentenceTransformer
from typing import List
import numpy as np

MODEL_NAME = "all-MiniLM-L6-v2"

In [6]:
def generate_real_estate_listings():
    """
    Generates 12 diverse real estate listings using an LLM.
    """
    prompt = """
    Generate 12 realistic real estate listings in JSON format, using locations from Singapore, ensuring each listing contains the following:
    - `index` (integer)
    - `Neighborhood` (string)
    - `Price` (string, formatted as $XXX,XXX)
    - `Bedrooms` (integer)
    - `Bathrooms` (integer)
    - `House Size` (string, formatted as X,XX sqft)
    - `Description` (detailed string)
    - `Neighborhood Description` (detailed string)

    Ensure correct JSON format and diverse property types.
    """
    response = llm(prompt)  # Call the LLM to generate listings
    return json.loads(response)  # Convert string to JSON format

In [7]:
def save_listings_to_file(listings, filename="listings.txt"):
    """
    Saves the generated listings to a JSON file.
    """
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(listings, file, indent=4)

def read_json_listings(file_path):
    """
    Reads listings from a JSON file.
    """
    with open(file_path, "r", encoding="utf-8") as fh:
        return json.load(fh)  # Ensure it loads as a list of dicts

def generate_embeddings(input_data: List[str]) -> np.ndarray:
    """
    Generates embeddings for the given listings using Sentence Transformers.
    """
    model = SentenceTransformer(MODEL_NAME)
    return model.encode(input_data)

# Generate and store listings
listings = generate_real_estate_listings()
save_listings_to_file(listings)

In [8]:
# Read listings from file
listings = read_json_listings("listings.txt")

# Extract listing descriptions for embedding
descriptions = [listing["Description"] for listing in listings]

# Generate embeddings
embeddings = generate_embeddings(descriptions)

In [9]:
# Print the generated listings
print("Generated Real Estate Listings:")
print(listings)

Generated Real Estate Listings:
[{'index': 1, 'Neighborhood': 'Orchard', 'Price': '$1,200,000', 'Bedrooms': 2, 'Bathrooms': 2, 'House Size': '1,000 sqft', 'Description': 'Modern 2-bedroom apartment in the heart of Orchard Road. Fully furnished with high-end appliances and a balcony with city views.', 'Neighborhood Description': 'Orchard is known for its upscale shopping malls, fine dining restaurants, and vibrant nightlife.'}, {'index': 2, 'Neighborhood': 'Tanjong Pagar', 'Price': '$900,000', 'Bedrooms': 1, 'Bathrooms': 1, 'House Size': '700 sqft', 'Description': 'Cozy 1-bedroom shophouse apartment in Tanjong Pagar. Renovated with a modern kitchen and a spacious living area.', 'Neighborhood Description': 'Tanjong Pagar is a trendy neighborhood with a mix of heritage shophouses, hip cafes, and bustling bars.'}, {'index': 3, 'Neighborhood': 'Sentosa Cove', 'Price': '$5,000,000', 'Bedrooms': 4, 'Bathrooms': 4, 'House Size': '3,500 sqft', 'Description': 'Luxurious 4-bedroom waterfront vill

## Semantic Search

#### Creating a Vector Database and Storing Listings
create a vector database and successfully store real estate listing embeddings within it. The database should effectively store and organize the embeddings generated from the LLM-created listings.

In [10]:
model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = OpenAI(model_name=model_name, temperature=temperature, max_tokens = 900)

In [11]:
import json
import numpy as np
import chromadb
from typing import List, Dict

# Define the model to use for generating embeddings
MODEL_NAME = 'paraphrase-MiniLM-L6-v2'

In [12]:
# Function to read and parse JSON listings from a text file
def read_json_listings(file_path: str) -> List[Dict]:
    """
    Reads listings from a JSON-formatted text file.
    
    Parameters:
    file_path (str): Path to the JSON file containing listings.
    
    Returns:
    List[Dict]: A list of listing dictionaries.
    """
    with open(file_path, "r", encoding="utf-8") as fh:
        return json.load(fh)  # Load JSON directly

In [13]:
# Function to generate embeddings for listings
def generate_embeddings(input_data: List[str]) -> np.ndarray:
    """
    Generates embeddings for the given listings using Sentence Transformers.
    
    Parameters:
    input_data (List[str]): A list of listing descriptions.
    
    Returns:
    np.ndarray: Array of embeddings.
    """
    model = SentenceTransformer(MODEL_NAME)
    return model.encode(input_data)

# Read and process listings
listings = read_json_listings("listings.txt")
# Extract listing descriptions for embedding
descriptions = [listing["Description"] for listing in listings]

# Generate embeddings
embeddings = generate_embeddings(descriptions)

In [14]:
# Ensure embeddings and listings match in length
assert len(embeddings) == len(listings), "Mismatch between embeddings and listings length!"

# Save embeddings for later use
np.save("listings_embeddings.npy", embeddings)

In [15]:
# Initialize ChromaDB client
client = chromadb.PersistentClient(path="./chroma_real_estate")

# Define collection name
collection_name = "real_estate_listings"

# Check if collection exists and delete before re-adding to prevent duplicates
existing_collections = client.list_collections()
if collection_name in [col.name for col in existing_collections]:
    client.delete_collection(name=collection_name)
    
# Create or get the collection
collection = client.get_or_create_collection(name=collection_name)

old one

In [16]:
# Prepare data for insertion
ids = [f"listing_{i}" for i in range(len(listings))]
metadata = listings  # Store full listings as metadata

# Add data to ChromaDB
collection.add(
    ids=ids,
    embeddings=embeddings.tolist(),  # Convert numpy array to list
    metadatas=metadata
)

new one

In [17]:
# Prepare data for insertion
ids = [f"listing_{i}" for i in range(len(listings))]
metadata = [{
    "index": listing.get("index", "N/A"),
    "Neighborhood": listing.get("Neighborhood", "Unknown"),
    "Price": listing.get("Price", "N/A"),
    "Bedrooms": listing.get("Bedrooms", "N/A"),
    "Bathrooms": listing.get("Bathrooms", "N/A"),
    "House Size": listing.get("House Size", "N/A"),
    "Description": listing.get("Description", "No description available."),
    "Neighborhood Description": listing.get("Neighborhood Description", "No neighborhood description available.")
} for listing in listings]

# Add data to ChromaDB
collection.add(
    ids=ids,
    embeddings=embeddings.tolist(),  # Convert numpy array to list
    metadatas=metadata
)

Insert of existing embedding ID: listing_0
Insert of existing embedding ID: listing_1
Insert of existing embedding ID: listing_2
Insert of existing embedding ID: listing_3
Insert of existing embedding ID: listing_4
Insert of existing embedding ID: listing_5
Insert of existing embedding ID: listing_6
Insert of existing embedding ID: listing_7
Insert of existing embedding ID: listing_8
Insert of existing embedding ID: listing_9
Insert of existing embedding ID: listing_10
Insert of existing embedding ID: listing_11
Add of existing embedding ID: listing_0
Add of existing embedding ID: listing_1
Add of existing embedding ID: listing_2
Add of existing embedding ID: listing_3
Add of existing embedding ID: listing_4
Add of existing embedding ID: listing_5
Add of existing embedding ID: listing_6
Add of existing embedding ID: listing_7
Add of existing embedding ID: listing_8
Add of existing embedding ID: listing_9
Add of existing embedding ID: listing_10
Add of existing embedding ID: listing_11


In [18]:
# Retrieve and verify stored data
retrieved_data = collection.get()
print("\n✅ Retrieved data sample:")
print(json.dumps(retrieved_data['metadatas'][:1], indent=4))  # Print first listing metadata
print(f"✅ Inserted {len(ids)} listings into '{collection_name}' collection.")


✅ Retrieved data sample:
[
    {
        "Bathrooms": 2,
        "Bedrooms": 2,
        "Description": "Modern 2-bedroom apartment in the heart of Orchard Road. Fully furnished with high-end appliances and a balcony with city views.",
        "House Size": "1,000 sqft",
        "Neighborhood": "Orchard",
        "Neighborhood Description": "Orchard is known for its upscale shopping malls, fine dining restaurants, and vibrant nightlife.",
        "Price": "$1,200,000",
        "index": 1
    }
]
✅ Inserted 12 listings into 'real_estate_listings' collection.


#### Semantic Search of Listings Based on Buyer Preferences

The application must include a functionality where listings are semantically searched based on given buyer preferences. The search should return listings that closely match the input preferences.

In [19]:
!pip install rapidfuzz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable


In [20]:
import json
import re
import numpy as np
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from rapidfuzz import process

# Load the model
MODEL_NAME = "all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)

# Initialize ChromaDB client
client = chromadb.PersistentClient(path="./chroma_real_estate")
collection_name = "real_estate_listings"
collection = client.get_or_create_collection(name=collection_name)

In [21]:
def extract_keywords(text):
    """Extract key nouns and adjectives while removing stopwords."""
    text = text.lower()
    words = re.findall(r'\b[a-zA-Z]+\b', text)
    return set(word for word in words if word not in ENGLISH_STOP_WORDS)

def infer_budget(query):
    """Infer budget from query based on price-related terms."""
    price_mapping = {
        "cheap": 500000,
        "affordable": 800000,
        "budget-friendly": 1000000,
        "mid-range": 2000000,
        "luxury": 5000000,
        "high-end": 10000000
    }
    for term, price in price_mapping.items():
        if term in query.lower():
            return price
    return None

def extract_query_components(query):
    """Extract structured components like bedrooms, budget, and features."""
    components = {
        "bedrooms": re.search(r'(\d+)\s*-?\s*bedroom', query, re.IGNORECASE),
        "features": re.findall(r'(gym|pool|garden|balcony|modern kitchen|public transportation)', query, re.IGNORECASE),
        "budget": re.search(r'\$?(\d+[mk]?)', query, re.IGNORECASE)
    }
    return {
        "bedrooms": int(components["bedrooms"].group(1)) if components["bedrooms"] else None,
        "features": components["features"] if components["features"] else [],
        "budget": int(components["budget"].group(1).replace('m', '000000').replace('k', '000')) if components["budget"] else infer_budget(query)
    }

def fuzzy_match_neighborhood(query, neighborhoods):
    """Use fuzzy search to find the closest matching neighborhood."""
    match, score = process.extractOne(query, neighborhoods)
    return match if score > 70 else None  # Threshold to ensure a good match

def semantic_search(query):
    """Perform semantic search based on the query."""
    query_embedding = model.encode([query]).tolist()
    query_results = collection.query(query_embeddings=query_embedding, n_results=5)

    query_components = extract_query_components(query)
    query_keywords = extract_keywords(query)
    
    print("\n🏡 **Top Matching Listings:**\n")

    for i, metadata in enumerate(query_results["metadatas"][0]):  # Correct metadata retrieval
        if metadata:
            print(f"🔹 **Listing {i+1}**")
            print(f"🏙️  **Neighborhood:** {metadata.get('Neighborhood', 'N/A')}")
            print(f"💰 **Price:** {metadata.get('Price', 'N/A')}")
            print(f"🛏️  **Bedrooms:** {metadata.get('Bedrooms', 'N/A')}")
            print(f"🛁 **Bathrooms:** {metadata.get('Bathrooms', 'N/A')}")
            print(f"📏 **Size:** {metadata.get('House Size', 'N/A')}")
            print(f"📜 **Description:** {metadata.get('Description', 'N/A')}")
            print("-" * 50)

    return query_results

In [22]:
# Example query: "Looking for a 3-bedroom condo with a balcony in Orchard for around $2M"
query = "Looking for a 3-bedroom condo with a balcony in Orchard for around $2M"
semantic_search(query)

Add of existing embedding ID: listing_0
Add of existing embedding ID: listing_1
Add of existing embedding ID: listing_2
Add of existing embedding ID: listing_3
Add of existing embedding ID: listing_4
Add of existing embedding ID: listing_5
Add of existing embedding ID: listing_6
Add of existing embedding ID: listing_7
Add of existing embedding ID: listing_8
Add of existing embedding ID: listing_9
Add of existing embedding ID: listing_10
Add of existing embedding ID: listing_11



🏡 **Top Matching Listings:**

🔹 **Listing 1**
🏙️  **Neighborhood:** Jurong East
💰 **Price:** $700,000
🛏️  **Bedrooms:** 2
🛁 **Bathrooms:** 2
📏 **Size:** 900 sqft
📜 **Description:** Well-maintained 2-bedroom HDB flat in Jurong East. Spacious layout, natural light, and close to shopping malls and amenities.
--------------------------------------------------
🔹 **Listing 2**
🏙️  **Neighborhood:** Orchard
💰 **Price:** $1,200,000
🛏️  **Bedrooms:** 2
🛁 **Bathrooms:** 2
📏 **Size:** 1,000 sqft
📜 **Description:** Modern 2-bedroom apartment in the heart of Orchard Road. Fully furnished with high-end appliances and a balcony with city views.
--------------------------------------------------
🔹 **Listing 3**
🏙️  **Neighborhood:** Sengkang
💰 **Price:** $800,000
🛏️  **Bedrooms:** 4
🛁 **Bathrooms:** 3
📏 **Size:** 1,200 sqft
📜 **Description:** Spacious 4-bedroom executive apartment in Sengkang. High floor, unblocked views, and close to LRT station and amenities.
---------------------------------------

{'ids': [['listing_8', 'listing_0', 'listing_10', 'listing_11', 'listing_1']],
 'distances': [[20.733030802767452,
   23.05306944622186,
   26.573304797626555,
   26.842813796863357,
   29.514678931461496]],
 'metadatas': [[{'Bathrooms': 2,
    'Bedrooms': 2,
    'Description': 'Well-maintained 2-bedroom HDB flat in Jurong East. Spacious layout, natural light, and close to shopping malls and amenities.',
    'House Size': '900 sqft',
    'Neighborhood': 'Jurong East',
    'Neighborhood Description': 'Jurong East is a bustling residential and commercial hub with shopping centers, parks, and entertainment options.',
    'Price': '$700,000',
    'index': 9},
   {'Bathrooms': 2,
    'Bedrooms': 2,
    'Description': 'Modern 2-bedroom apartment in the heart of Orchard Road. Fully furnished with high-end appliances and a balcony with city views.',
    'House Size': '1,000 sqft',
    'Neighborhood': 'Orchard',
    'Neighborhood Description': 'Orchard is known for its upscale shopping malls, f

In [23]:
# Allow user input
user_query = input("\nDescribe your ideal home: ")
semantic_search(user_query)


Describe your ideal home: looking for a house for 4, with a cozy environment. I would like it to be affordable.

🏡 **Top Matching Listings:**

🔹 **Listing 1**
🏙️  **Neighborhood:** Jurong East
💰 **Price:** $700,000
🛏️  **Bedrooms:** 2
🛁 **Bathrooms:** 2
📏 **Size:** 900 sqft
📜 **Description:** Well-maintained 2-bedroom HDB flat in Jurong East. Spacious layout, natural light, and close to shopping malls and amenities.
--------------------------------------------------
🔹 **Listing 2**
🏙️  **Neighborhood:** Orchard
💰 **Price:** $1,200,000
🛏️  **Bedrooms:** 2
🛁 **Bathrooms:** 2
📏 **Size:** 1,000 sqft
📜 **Description:** Modern 2-bedroom apartment in the heart of Orchard Road. Fully furnished with high-end appliances and a balcony with city views.
--------------------------------------------------
🔹 **Listing 3**
🏙️  **Neighborhood:** Sengkang
💰 **Price:** $800,000
🛏️  **Bedrooms:** 4
🛁 **Bathrooms:** 3
📏 **Size:** 1,200 sqft
📜 **Description:** Spacious 4-bedroom executive apartment in Sengk

{'ids': [['listing_8', 'listing_0', 'listing_10', 'listing_11', 'listing_1']],
 'distances': [[21.740526375321302,
   23.92233766561755,
   27.19485010532861,
   27.284288557563425,
   28.82004357892665]],
 'metadatas': [[{'Bathrooms': 2,
    'Bedrooms': 2,
    'Description': 'Well-maintained 2-bedroom HDB flat in Jurong East. Spacious layout, natural light, and close to shopping malls and amenities.',
    'House Size': '900 sqft',
    'Neighborhood': 'Jurong East',
    'Neighborhood Description': 'Jurong East is a bustling residential and commercial hub with shopping centers, parks, and entertainment options.',
    'Price': '$700,000',
    'index': 9},
   {'Bathrooms': 2,
    'Bedrooms': 2,
    'Description': 'Modern 2-bedroom apartment in the heart of Orchard Road. Fully furnished with high-end appliances and a balcony with city views.',
    'House Size': '1,000 sqft',
    'Neighborhood': 'Orchard',
    'Neighborhood Description': 'Orchard is known for its upscale shopping malls, fin

## Augmented Response Generation

#### Logic for Searching and Augmenting Listing Descriptions

The project must demonstrate a logical flow where buyer preferences are used to search and then augment the description of real estate listings. The augmentation should personalize the listing without changing factual information.

In [24]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# Initialize OpenAI LLM
model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = OpenAI(model_name=model_name, temperature=temperature, max_tokens=900)

In [25]:
# Chain of Thought Examples for Personalizing Listings
example1 = {
    "buyer_preferences": "Looking for a modern 3-bedroom home with a large backyard for children and a pet-friendly neighborhood.",
    "listing": "Spacious 3-bedroom house with an open-plan kitchen and a cozy living room.",
    "personalized_listing": """This modern 3-bedroom home is perfect for a growing family. It features a spacious open-plan kitchen where you can prepare meals while keeping an eye on the kids. The backyard is large, ideal for outdoor activities, BBQ nights, and a safe space for pets to roam. The neighborhood is pet-friendly and includes nearby parks and walking trails for a comfortable, active lifestyle."""
}

example2 = {
    "buyer_preferences": "Luxury apartment with a city view, close to public transport, ideal for a professional working in downtown.",
    "listing": "High-rise condo with a balcony and modern amenities.",
    "personalized_listing": """Experience the ultimate urban lifestyle in this luxurious high-rise condo, designed for professionals who seek convenience and elegance. The unit boasts a stunning city view from its private balcony, allowing you to unwind after a busy workday. Located just minutes away from major public transport hubs, this residence ensures a hassle-free commute to downtown offices, restaurants, and entertainment centers."""
}

# Define Prompt Template
example_prompt = PromptTemplate(
    input_variables=["buyer_preferences", "listing", "personalized_listing"],
    template="Buyer Preferences: {buyer_preferences}\nOriginal Listing: {listing}\nPersonalized Listing: {personalized_listing}\n"
)

# Create Few-Shot Prompt Template
few_shot_prompt = FewShotPromptTemplate(
    examples=[example1, example2],
    example_prompt=example_prompt,
    suffix="Buyer Preferences: {buyer_preferences}\nOriginal Listing: {listing}\nPersonalized Listing:",
    input_variables=["buyer_preferences", "listing"]
)

In [26]:

# Function to detect and correct factual inaccuracies
def correct_factual_inaccuracies(original_listing: str, personalized_listing: str) -> str:
    correction_prompt = f"""
    Original Listing: {original_listing}
    Personalized Listing: {personalized_listing}
    
    Ensure the personalized listing is factually accurate based on the original listing. Correct any inaccuracies while keeping it engaging.
    
    Corrected Personalized Listing:
    """
    response = llm(correction_prompt).strip()
    return response

# Function to personalize a listing with factual accuracy
def personalize_listing(buyer_preferences: str, retrieved_listings: list):
    """
    Personalizes real estate listings based on buyer's preferences using Chain of Thought reasoning.
    Ensures factual accuracy by detecting and correcting inaccuracies.
    
    Parameters:
    - buyer_preferences (str): Buyer's specific preferences.
    - retrieved_listings (list): List of top matching listings.
    
    Returns:
    - List of personalized listings.
    """
    personalized_listings = []
    
    for listing in retrieved_listings:
        prompt_text = few_shot_prompt.format(
            buyer_preferences=buyer_preferences,
            listing=listing["listing"]
        )
        personalized_description = llm(prompt_text).strip()
        
        # Limit the number of correction iterations
        max_iterations = 3
        for _ in range(max_iterations):
            corrected_description = correct_factual_inaccuracies(listing["listing"], personalized_description)
            if corrected_description == personalized_description:
                break  # Stop if no changes were made
            personalized_description = corrected_description
        
        personalized_listings.append({
            "original_listing": listing["listing"],
            "personalized_listing": personalized_description
        })
    
    return personalized_listings

In [27]:
# Query ChromaDB
def retrieve_listings(buyer_query: str, n_results=3):
    """Retrieves the top matching real estate listings from ChromaDB."""
    embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    query_embeddings = embedding_model.encode(buyer_query).tolist()
    results = collection.query(query_embeddings=[query_embeddings], n_results=n_results)
    return results.get("metadatas", [])

#### Use of LLM for Generating Personalized Descriptions

The submission must utilize an LLM to generate personalized descriptions for the real estate listings based on buyer preferences. The descriptions should be unique, appealing, and tailored to the preferences provided.

In [28]:
def personalize_listing(buyer_preferences: str, retrieved_listings: list):
    """
    Personalizes real estate listings based on buyer's preferences using Chain of Thought reasoning.
    Ensures factual accuracy by detecting and correcting inaccuracies.
    
    Parameters:
    - buyer_preferences (str): Buyer's specific preferences.
    - retrieved_listings (list): List of top matching listings.
    
    Returns:
    - List of personalized listings.
    """
    # Extract actual listing data from nested list if necessary
    if retrieved_listings and isinstance(retrieved_listings[0], list):
        retrieved_listings = retrieved_listings[0]  # Unwrap the nested list

    personalized_listings = []
    
    for listing in retrieved_listings:
        # Create a structured listing description with all relevant details
        full_listing_details = f"""
        Neighborhood: {listing['Neighborhood']}
        Description: {listing['Description']}
        Bedrooms: {listing['Bedrooms']}
        Bathrooms: {listing['Bathrooms']}
        House Size: {listing['House Size']}
        Price: {listing['Price']}
        Nearby: {listing['Neighborhood Description']}
        """

        # Updated prompt template to prevent factual errors
        prompt_text = f"""
        You are a real estate expert. Personalize the property listing to make it more appealing based on the buyer's preferences. 
        However, you **MUST NOT** alter factual details such as the number of bedrooms, bathrooms, size, price, or location.
        
        Buyer Preferences: {buyer_preferences}
        Original Listing:
        {full_listing_details}
        
        Personalized Listing:
        """
        
        personalized_description = llm(prompt_text).strip()

        # Post-processing step: Ensure no factual changes were made
        factual_corrections = correct_factual_inaccuracies(full_listing_details, personalized_description)

        personalized_listings.append({
            "original_listing": full_listing_details,
            "personalized_listing": factual_corrections
        })
    
    return personalized_listings

In [29]:
# Example Usage
buyer_query = input("Please describe your ideal house. You may share details on your preferred neighborhood, number of rooms, amenities, or more.\n")
retrieved_listings = retrieve_listings(buyer_query)

# Generate personalized listings
personalized_results = personalize_listing(buyer_query, retrieved_listings)

# Print results
print("\n=== Personalized Real Estate Listings ===\n")
for idx, result in enumerate(personalized_results, 1):
    print(f"{idx}. **Original Listing:** {result['original_listing']}\n")
    print(f"   **Personalized Listing:** {result['personalized_listing']}\n")

Please describe your ideal house. You may share details on your preferred neighborhood, number of rooms, amenities, or more.
looking for a house for 4, with a cozy environment. I would like it to be affordable.

=== Personalized Real Estate Listings ===

1. **Original Listing:** 
        Neighborhood: Holland Village
        Description: Charming 4-bedroom townhouse in Holland Village. Spacious living area, private garden, and a rooftop terrace.
        Bedrooms: 4
        Bathrooms: 3
        House Size: 2,000 sqft
        Price: $2,300,000
        Nearby: Holland Village is a vibrant expat-friendly neighborhood with trendy cafes, boutiques, and a bustling market.
        

   **Personalized Listing:** Neighborhood: Holland Village
Description: Welcome to your cozy oasis in the heart of Holland Village! This charming 4-bedroom townhouse offers a warm and inviting atmosphere, perfect for creating lasting memories with your family. Enjoy the spacious living area, private garden for outd